In [ ]:
!pip install --upgrade pip
!pip install flask

In [6]:
from flask import Flask, request, render_template, jsonify
import pandas as pd
import boto3

In [ ]:
# Create Flask application
app = Flask(__name__)

# Set up the SageMaker runtime
client = boto3.client('sagemaker-runtime')

# The name of the endpoint we created
ENDPOINT_NAME = 'xgboost-2023-10-24-20-00-48-433'

@app.route('/')
def home():
    """
    This function just responds to the browser URL
    localhost:5000/
    :return:        the rendered template 'index.html'
    """
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Step 1: Extract data from form
        form_data = request.form
        track_name = form_data['track_name']
        artist_name = form_data['artist_name']
        artist_count = form_data['artist_count']
        released_year = form_data['released_year']
        released_month = form_data['released_month']
        released_day = form_data['released_day']
        in_spotify_playlists = form_data['in_spotify_playlists']
        in_spotify_charts = form_data['in_spotify_charts']
        in_apple_playlists = form_data['in_apple_playlists']
        in_apple_charts = form_data['in_apple_charts']
        playlist_adds = form_data['playlist_adds']
        bpm = form_data['bpm']
        key = form_data['key']
        mode = form_data['mode']
        danceability = form_data['danceability']
        valence = form_data['valence']
        energy = form_data['energy']
        acousticness = form_data['acousticness']
        instrumentalness = form_data['instrumentalness']
        liveness = form_data['liveness']
        speechiness = form_data['speechiness']

        # Step 2: Process data (as necessary)
        # If the model requires numeric inputs, ensure each one is converted from string
        artist_count = int(artist_count)
        released_year = int(released_year)
        released_month = int(released_month)
        released_day = int(released_day)
        in_spotify_playlists = int(in_spotify_playlists)
        in_spotify_charts = int(in_spotify_charts)
        in_apple_playlists = int(in_apple_playlists)
        in_apple_charts = int(in_apple_charts)
        playlist_adds = int(playlist_adds)
        bpm = int(bpm)
        # For the following, convert to float as they seem to represent percentages/ratios
        danceability = float(danceability)
        valence = float(valence)
        energy = float(energy)
        acousticness = float(acousticness)
        instrumentalness = float(instrumentalness)
        liveness = float(liveness)
        speechiness = float(speechiness)

        # Step 3: Organize data into expected format (e.g., a DataFrame)
        data_for_prediction = {
            'track_name': [track_name],
            'artist_name': [artist_name],
            'artist_count': [artist_count],
            'released_year': [released_year],
            'released_month': [released_month],
            'released_day': [released_day],
            'in_spotify_playlists': [in_spotify_playlists],
            'in_spotify_charts': [in_spotify_charts],
            'in_apple_playlists': [in_apple_playlists],
            'in_apple_charts': [in_apple_charts],
            'playlist_adds': [playlist_adds],
            'bpm': [bpm],
            'key': [key],
            'mode': [mode],
            'danceability_%': [danceability],
            'valence_%': [valence],
            'energy_%': [energy],
            'acousticness_%': [acousticness],
            'instrumentalness_%': [instrumentalness],
            'liveness_%': [liveness],
            'speechiness_%': [speechiness]
        }
        data_df = pd.DataFrame(data_for_prediction)
        
        # Select only numeric columns
        numeric_data_df = data_df.select_dtypes(include = [np.number])

        # Convert the DataFrame to CSV since the endpoint expects CSV input
        body = numeric_data_df.to_csv(index=False, header=False)  # Convert DataFrame to CSV

        # Step 4: Get prediction from model
        response = client.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            Body=body,
            ContentType='text/csv'
        )

        # Unpack the response
        result = response['Body'].read().decode()

        # Step 5: Return results
        # You might want to parse or process the result before returning it, depending on what it looks like
        return render_template('results.html', prediction=result)  # replace with your actual results file name

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
